## Esercizio 6.1

In [5]:
%pip install ipython-sql==0.5.0

*Load the SQL extension*

In [6]:
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [8]:
%sql sqlite:///my_database.db

*Creating the tables*

In [9]:
%%sql

CREATE TABLE departments (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    location TEXT
);

CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    department_id INTEGER,
    hire_date TEXT,  -- SQLite does not have type DATE
    salary REAL,
    bonus REAL,
    job_title TEXT,
    manager_id INTEGER,
    performance_rating INTEGER,
    email TEXT,
    phone_number TEXT,
    FOREIGN KEY (department_id) REFERENCES departments(id),
    FOREIGN KEY (manager_id) REFERENCES employees(id) -- self-referencing foreign key: every manager is also an employees, therefore manager_id is the manager's employee id.
);

INSERT INTO departments (id, name, location) VALUES
(1, 'Engineering', 'New York'),
(2, 'Sales', 'Los Angeles'),
(3, 'HR', 'Chicago'),
(4, 'Marketing', 'San Francisco'),
(5, 'Finance', 'Boston');

INSERT INTO employees (id, first_name, last_name, department_id, hire_date, salary, bonus, job_title, manager_id, performance_rating, email, phone_number) VALUES
(1, 'Alice', 'Smith',       1, '2010-06-01', 120000, 10000, 'CTO',                   NULL, 5, 'alice.smith@example.com', '555-0100'),
(2, 'Bob', 'Johnson',       1, '2012-09-15',  90000,  5000, 'Senior Engineer',       1, 4, 'bob.johnson@example.com', '555-0101'),
(3, 'Carol', 'Williams',    1, '2015-03-20',  80000,   NULL, 'Engineer',              2, 3, 'carol.williams@example.com', '555-0102'),
(4, 'David', 'Jones',       2, '2011-11-11',  95000,  7000, 'Sales Manager',         NULL, 4, 'david.jones@example.com', '555-0103'),
(5, 'Eva', 'Brown',         2, '2016-05-22',  70000,  3000, 'Sales Representative',  4, 3, 'eva.brown@example.com', '555-0104'),
(6, 'Frank', 'Davis',       2, '2018-07-01',  68000,   NULL, 'Sales Representative',  4, 2, 'frank.davis@example.com', '555-0105'),
(7, 'Grace', 'Miller',      3, '2013-02-28',  75000,  4000, 'HR Specialist',         NULL, 4, 'grace.miller@example.com', '555-0106'),
(8, 'Henry', 'Wilson',      3, '2019-10-10',  60000,  2000, 'HR Assistant',           7, 3, 'henry.wilson@example.com', '555-0107'),
(9, 'Ivy', 'Moore',         4, '2014-08-19',  85000,  5000, 'Marketing Manager',     NULL, 4, 'ivy.moore@example.com', '555-0108'),
(10, 'Sam', 'White',        NULL, '2023-01-01', 50000, NULL, 'Intern',                3, NULL, NULL, NULL),
(11, 'Jack', 'Taylor',      4, '2020-01-15',  65000,   NULL, 'Marketing Specialist',   9, 3, 'jack.taylor@example.com', '555-0109'),
(12, 'Kathy', 'Anderson',   5, '2017-04-03',  78000,  4500, 'Finance Analyst',       NULL, 4, 'kathy.anderson@example.com', '555-0110'),
(13, 'Leo', 'Thomas',       5, '2021-12-05',  55000,   NULL, 'Junior Finance Analyst',11, 3, 'leo.thomas@example.com', '555-0111');

 * sqlite:///my_database.db
Done.
Done.
5 rows affected.
13 rows affected.


[]

## 1.

Write a query to find first name and last name of employees whose first name: has 3 letters; ends in 'a'.

In [10]:
%%sql
SELECT first_name, last_name
FROM employees
WHERE LENGTH(first_name) = 3
AND first_name LIKE '__a';

 * sqlite:///my_database.db
Done.


first_name,last_name
Eva,Brown


## 2.

Write a query to find, for each department: the name of the department; the number of employees; the number of employees who received a bonus; the total amount of bonuses disbursed.

In [17]:
%%sql 
SELECT d.name AS department_name, 
       COUNT(e.id) AS total_employees,
       COUNT(e.bonus) AS employees_with_bonus,
       COALESCE(SUM(e.bonus), 0) AS total_bonus_disbursed
FROM departments d
LEFT JOIN employees e ON d.id = e.department_id
GROUP BY d.name

 * sqlite:///my_database.db
Done.


department_name,total_employees,employees_with_bonus,total_bonus_disbursed
Engineering,3,2,15000.0
Finance,2,1,4500.0
HR,2,2,6000.0
Marketing,2,1,5000.0
Sales,3,2,10000.0


## 3.

Write a query to find employees who received the same bonus as at least one other employee.

In [18]:
%%sql
SELECT e1.id, e1.first_name, e1.last_name, e1.bonus
FROM employees e1
JOIN employees e2 ON e1.bonus = e2.bonus 
                  AND e1.id <> e2.id
WHERE e1.bonus IS NOT NULL

 * sqlite:///my_database.db
Done.


id,first_name,last_name,bonus
2,Bob,Johnson,5000.0
9,Ivy,Moore,5000.0
